In [ ]:
!pip install pyserini
!pip install dice-ml
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 93.3 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import pyserini
import os
from pyserini.search import get_topics
from pyserini.search import LuceneSearcher
import math
import ast
import json
from pyserini.index.lucene import IndexReader
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, ENGLISH_STOP_WORDS
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from spacy.lang.de.stop_words import STOP_WORDS
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
import dice_ml
from dice_ml.utils import helpers
import numpy as np
import random
import warnings

warnings.filterwarnings('ignore')

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

/usr/local/lib/python3.10/dist-packages/dice_ml/utils/exception.py:12: UserWarning: UserConfigValidationException will be deprecated from dice_ml.utils. Please import UserConfigValidationException from raiutils.exceptions.
  warnings.warn("UserConfigValidationException will be deprecated from dice_ml.utils. "


In [ ]:
topics = get_topics('msmarco-passage-dev-subset')

In [ ]:
topics

{1102330: {'title': 'why do people grind teeth in sleep'},
 160885: {'title': 'do you need immunizations for belize'},
 1091115: {'title': 'recurrent gastric cancer symptoms'},
 613179: {'title': 'what county is stafford tx'},
 1091116: {'title': 'recommended temperature'},
 1092441: {'title': 'most romantic zodiac sign'},
 1091112: {'title': 'redux phone number'},
 1093773: {'title': 'jefferson college address'},
 867490: {'title': 'what kind of doctor is dr. nassar'},
 31595: {'title': 'average car bill in florida'},
 54235: {'title': 'black death effects on europe culture'},
 1091108: {'title': 'refers to what happens when sensory information is detected by a sensory receptor'},
 306105: {'title': 'how much do jockeys have to weigh'},
 914321: {'title': 'what type of rna carries the information that specifies a protein?'},
 1102335: {'title': 'why do people buy cars'},
 1066161: {'title': 'what is a tool used to'},
 93823: {'title': 'co-curricular vs extracurricular definition'},
 8

In [ ]:
random_topics = random.sample(topics.items(), 10)
random_topics = [(id, title['title']) for id, title in random_topics]
queries = pd.DataFrame(random_topics, columns=['query_id', 'query'])

queries

<ipython-input-69-128c12aa9fe2>:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_topics = random.sample(topics.items(), 10)


,query_id,query
0,992839,definition of curfew
1,1013965,"what type of wave is electromagnetic,"
2,1090730,shopko kennewick address
3,872869,what law repealed prohibition
4,1046463,what is sealaska corporation
5,1024312,who are the months named after
6,1029492,what is transient global am
7,676454,what is a caret?
8,992559,which vitamins help heal bruises
9,844658,what is the role of lipids in a cell


In [ ]:
searcher = LuceneSearcher.from_prebuilt_index('msmarco-passage')

In [ ]:
K = 30 # Top K words used for generating vocabulary and feature vector (ranking and used for classifier)

T = 10 #Threshold for top T words labelled as 1 and rest as 0.



In [ ]:
import json
jsondoc = []
for q in range(0,10):
  query = queries['query'][q]
  hits = searcher.search(query,K)
  random_numbers = random.sample(range(T, K), 5)
  for j in random_numbers:
    jsondoc.append((j+1,hits[j].score,json.loads(hits[j].raw)))
jsondoc

[(24,
  8.082300186157227,
  {'id': '3807532',
   'contents': 'What curfew rules should you set for your teenager? A curfew sets the expectation of when your teen will be home, for their safety and out of respect for the rest of the family. You will also need to set and enforce the consequences for breaking the curfew agreement. Each family will have their own needs for setting limits.'}),
 (17,
  8.265999794006348,
  {'id': '8191267',
   'contents': 'An additional 6 percent, or 23 cities, were considering adopting curfew legislation, according to the survey. 50 Cities that have enacted new curfew ordinances or have amended existing curfew legislation since 1994 include Arlington, VA; Austin, TX; Baltimore, MD; Buffalo, NY; Phoenix, AZ; Oklahoma City, OK; and San Jose, CA. 51. Curfew laws vary with respect to the locale affected, timeframe, and sanctions. Most restrict minors to their homes or property between the hours of 11 p.m. and 6 a.m., with some jurisdictions allowing exceptions

In [ ]:
formatted_data = []
for rank, score, info_dict in jsondoc:
    formatted_data.append({
        'Rank': rank,
        'Score': score,
        'Doc_id': info_dict['id'],
        'Document': info_dict['contents']
    })

df = pd.DataFrame(formatted_data)
df

,Rank,Score,Doc_id,Document
0,24,8.082300,3807532,What curfew rules should you set for your teen...
1,17,8.266000,8191267,"An additional 6 percent, or 23 cities, were co..."
2,12,8.602500,5207344,The definition of universal is relating to or ...
3,26,8.044699,3807525,Curfew rules can be adjusted as your teen grow...
4,19,8.183300,2620523,Discuss is defined as to talk about and to con...
5,15,12.856800,2646538,FIND OUT MORE. Many different kinds of energy ...
6,19,12.720400,6615903,The electromagnetic spectrum contains many typ...
7,24,12.665900,752147,Sound waves and electromagnetic waves are diff...
8,17,12.729500,1742690,"In transverse waves, the oscillations (vibrati..."
9,18,12.723700,1839,Difference Between Electromagnetic Waves and R...


In [ ]:
queries_new = queries.loc[queries.index.repeat(5)].reset_index(drop=True)
queries_new

,query_id,query
0,992839,definition of curfew
1,992839,definition of curfew
2,992839,definition of curfew
3,992839,definition of curfew
4,992839,definition of curfew
5,1013965,"what type of wave is electromagnetic,"
6,1013965,"what type of wave is electromagnetic,"
7,1013965,"what type of wave is electromagnetic,"
8,1013965,"what type of wave is electromagnetic,"
9,1013965,"what type of wave is electromagnetic,"


In [ ]:
result = pd.concat([queries_new, df], axis=1).reset_index(drop=True)

result

,query_id,query,Rank,Score,Doc_id,Document
0,992839,definition of curfew,24,8.082300,3807532,What curfew rules should you set for your teen...
1,992839,definition of curfew,17,8.266000,8191267,"An additional 6 percent, or 23 cities, were co..."
2,992839,definition of curfew,12,8.602500,5207344,The definition of universal is relating to or ...
3,992839,definition of curfew,26,8.044699,3807525,Curfew rules can be adjusted as your teen grow...
4,992839,definition of curfew,19,8.183300,2620523,Discuss is defined as to talk about and to con...
5,1013965,"what type of wave is electromagnetic,",15,12.856800,2646538,FIND OUT MORE. Many different kinds of energy ...
6,1013965,"what type of wave is electromagnetic,",19,12.720400,6615903,The electromagnetic spectrum contains many typ...
7,1013965,"what type of wave is electromagnetic,",24,12.665900,752147,Sound waves and electromagnetic waves are diff...
8,1013965,"what type of wave is electromagnetic,",17,12.729500,1742690,"In transverse waves, the oscillations (vibrati..."
9,1013965,"what type of wave is electromagnetic,",18,12.723700,1839,Difference Between Electromagnetic Waves and R...


In [ ]:
result.to_csv('/content/sample_data/dataset.csv')